In [24]:
import pandas as pd
from sklearn.preprocessing import *
one_hot_encoder = OneHotEncoder()
from sklearn.linear_model import *
from sklearn.model_selection import *
from sklearn.metrics import *

# neon 
from neon_db_push import *

# pkl
import joblib

In [25]:
# getting the db 
df = fetch_raw_data()

In [26]:
df["job_description_length"].unique()

array([1076, 1268, 1974, ...,  604, 1975,  723], shape=(2000,))

In [27]:
df.head()

,job_id,job_title,salary_usd,salary_currency,experience_level,employment_type,company_location,company_size,employee_residence,remote_ratio,required_skills,education_required,years_experience,industry,posting_date,application_deadline,job_description_length,benefits_score,company_name
0,AI00001,AI Research Scientist,90376,USD,SE,CT,China,M,China,50,"Tableau, PyTorch, Kubernetes, Linux, NLP",Bachelor,9,Automotive,2024-10-18,2024-11-07,1076,5.9,Smart Analytics
1,AI00002,AI Software Engineer,61895,USD,EN,CT,Canada,M,Ireland,100,"Deep Learning, AWS, Mathematics, Python, Docker",Master,1,Media,2024-11-20,2025-01-11,1268,5.2,TechCorp Inc
2,AI00003,AI Specialist,152626,USD,MI,FL,Switzerland,L,South Korea,0,"Kubernetes, Deep Learning, Java, Hadoop, NLP",Associate,2,Education,2025-03-18,2025-04-07,1974,9.4,Autonomous Tech
3,AI00004,NLP Engineer,80215,USD,SE,FL,India,M,India,50,"Scala, SQL, Linux, Python",PhD,7,Consulting,2024-12-23,2025-02-24,1345,8.6,Future Systems
4,AI00005,AI Consultant,54624,EUR,EN,PT,France,S,Singapore,100,"MLOps, Java, Tableau, Python",Master,0,Media,2025-04-15,2025-06-23,1989,6.6,Advanced Robotics


In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   job_id                  15000 non-null  object 
 1   job_title               15000 non-null  object 
 2   salary_usd              15000 non-null  int64  
 3   salary_currency         15000 non-null  object 
 4   experience_level        15000 non-null  object 
 5   employment_type         15000 non-null  object 
 6   company_location        15000 non-null  object 
 7   company_size            15000 non-null  object 
 8   employee_residence      15000 non-null  object 
 9   remote_ratio            15000 non-null  int64  
 10  required_skills         15000 non-null  object 
 11  education_required      15000 non-null  object 
 12  years_experience        15000 non-null  int64  
 13  industry                15000 non-null  object 
 14  posting_date            15000 non-null

In [29]:
# check for duplicate values
df.duplicated().sum()

np.int64(0)

In [30]:
# check for null values
df.isnull().sum()
# there are no missing values 


job_id                    0
job_title                 0
salary_usd                0
salary_currency           0
experience_level          0
employment_type           0
company_location          0
company_size              0
employee_residence        0
remote_ratio              0
required_skills           0
education_required        0
years_experience          0
industry                  0
posting_date              0
application_deadline      0
job_description_length    0
benefits_score            0
company_name              0
dtype: int64

In [31]:
df["job_title"].unique()

array(['AI Research Scientist', 'AI Software Engineer', 'AI Specialist',
       'NLP Engineer', 'AI Consultant', 'AI Architect',
       'Principal Data Scientist', 'Data Analyst',
       'Autonomous Systems Engineer', 'AI Product Manager',
       'Machine Learning Engineer', 'Data Engineer', 'Research Scientist',
       'ML Ops Engineer', 'Robotics Engineer', 'Head of AI',
       'Deep Learning Engineer', 'Data Scientist',
       'Machine Learning Researcher', 'Computer Vision Engineer'],
      dtype=object)

In [32]:
col_name = [
    'job_title', 'experience_level', 'employment_type',
    'company_location', 'company_size', 'employee_residence',
    'education_required', 'industry'
]
# created a list of col name to encoded into one hot 

encoded_cols = one_hot_encoder.fit_transform(df[col_name])
feature_name_for_cols = one_hot_encoder.get_feature_names_out(col_name)

encoded_df = pd.DataFrame(encoded_cols.toarray(),columns=feature_name_for_cols)
encoded_df.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 90 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   job_title_AI Architect                 15000 non-null  float64
 1   job_title_AI Consultant                15000 non-null  float64
 2   job_title_AI Product Manager           15000 non-null  float64
 3   job_title_AI Research Scientist        15000 non-null  float64
 4   job_title_AI Software Engineer         15000 non-null  float64
 5   job_title_AI Specialist                15000 non-null  float64
 6   job_title_Autonomous Systems Engineer  15000 non-null  float64
 7   job_title_Computer Vision Engineer     15000 non-null  float64
 8   job_title_Data Analyst                 15000 non-null  float64
 9   job_title_Data Engineer                15000 non-null  float64
 10  job_title_Data Scientist               15000 non-null  float64
 11  jo

In [33]:
# data that does not need to be changed
num_cols = [ 'remote_ratio', 'years_experience',
    'job_description_length', 'benefits_score',"salary_usd"]

# combine both the 
encoded_df[num_cols] = df[num_cols]
encoded_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 95 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   job_title_AI Architect                 15000 non-null  float64
 1   job_title_AI Consultant                15000 non-null  float64
 2   job_title_AI Product Manager           15000 non-null  float64
 3   job_title_AI Research Scientist        15000 non-null  float64
 4   job_title_AI Software Engineer         15000 non-null  float64
 5   job_title_AI Specialist                15000 non-null  float64
 6   job_title_Autonomous Systems Engineer  15000 non-null  float64
 7   job_title_Computer Vision Engineer     15000 non-null  float64
 8   job_title_Data Analyst                 15000 non-null  float64
 9   job_title_Data Engineer                15000 non-null  float64
 10  job_title_Data Scientist               15000 non-null  float64
 11  jo

In [34]:
# pushin to neon 
# pushin to neon 
result = push_df_to_neon(encoded_df,table_name="23408189_one_hot_encoding")

df data is pushed !!


In [35]:
encoded_df["salary_usd"]

0         90376
1         61895
2        152626
3         80215
4         54624
          ...  
14995     38604
14996     57811
14997    189490
14998     79461
14999     56481
Name: salary_usd, Length: 15000, dtype: int64

Model implimentation

In [36]:
x = encoded_df.drop("salary_usd",axis=1)
y = encoded_df["salary_usd"]

In [37]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [38]:
model = LinearRegression()
model.fit(X_train, y_train)


,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [39]:
y_pred = model.predict(X_test)
y_pred


array([153535.16629545, 162528.46277663, 134195.90412126, ...,
        94500.17770852,  22998.6770552 ,  33135.88559715], shape=(3000,))

In [40]:
print("R² Score:", r2_score(y_test, y_pred))


R² Score: 0.8498168256809358


In [41]:
joblib.dump(model, 'salary_predictor.pkl')


['salary_predictor.pkl']

In [42]:
joblib.dump(one_hot_encoder, 'dt_encoder.pkl')

['dt_encoder.pkl']

In [43]:
encoded_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 95 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   job_title_AI Architect                 15000 non-null  float64
 1   job_title_AI Consultant                15000 non-null  float64
 2   job_title_AI Product Manager           15000 non-null  float64
 3   job_title_AI Research Scientist        15000 non-null  float64
 4   job_title_AI Software Engineer         15000 non-null  float64
 5   job_title_AI Specialist                15000 non-null  float64
 6   job_title_Autonomous Systems Engineer  15000 non-null  float64
 7   job_title_Computer Vision Engineer     15000 non-null  float64
 8   job_title_Data Analyst                 15000 non-null  float64
 9   job_title_Data Engineer                15000 non-null  float64
 10  job_title_Data Scientist               15000 non-null  float64
 11  jo

In [44]:
encoded_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 95 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   job_title_AI Architect                 15000 non-null  float64
 1   job_title_AI Consultant                15000 non-null  float64
 2   job_title_AI Product Manager           15000 non-null  float64
 3   job_title_AI Research Scientist        15000 non-null  float64
 4   job_title_AI Software Engineer         15000 non-null  float64
 5   job_title_AI Specialist                15000 non-null  float64
 6   job_title_Autonomous Systems Engineer  15000 non-null  float64
 7   job_title_Computer Vision Engineer     15000 non-null  float64
 8   job_title_Data Analyst                 15000 non-null  float64
 9   job_title_Data Engineer                15000 non-null  float64
 10  job_title_Data Scientist               15000 non-null  float64
 11  jo

In [45]:
import pandas as pd
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# Use your encoded dataframe
# Example: encoded_df = pd.read_csv("encoded_data.csv")  ← if loading from file

# Define target and features
y = encoded_df['salary_usd']              # Target column
X = encoded_df.drop(columns=['salary_usd'])  # Features (drop target)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train XGBoost model
model = XGBRegressor(objective='reg:squarederror', random_state=42)
model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_test)

# Evaluate performance
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

# Print evaluation metrics
print("RMSE:", rmse)
print("R² Score:", r2)


RMSE: 21897.52205159296
R² Score: 0.8684619665145874


In [46]:
df["job_title"].unique()

array(['AI Research Scientist', 'AI Software Engineer', 'AI Specialist',
       'NLP Engineer', 'AI Consultant', 'AI Architect',
       'Principal Data Scientist', 'Data Analyst',
       'Autonomous Systems Engineer', 'AI Product Manager',
       'Machine Learning Engineer', 'Data Engineer', 'Research Scientist',
       'ML Ops Engineer', 'Robotics Engineer', 'Head of AI',
       'Deep Learning Engineer', 'Data Scientist',
       'Machine Learning Researcher', 'Computer Vision Engineer'],
      dtype=object)